In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import * 

In [0]:
%run ../utility/delta_merge

In [0]:
silver_data_source  = spark.read.table('lakehouse.`03_silver`.earthquake_events')


In [0]:
silver_data_source.display()

In [0]:
#TODO
# 1. for a given hour how many events occured and it s max and min magnitude 


In [0]:
seismic_activity = (
    silver_data_source
    .withColumn("activity_houe", F.date_trunc("hour", F.col("time")))
    .groupBy("activity_houe","net")
    .agg(
        F.count("hash_id").alias("total_event"),
        F.avg("mag").alias("avg_mag"),
        F.max("mag").alias("max_mag")
    )
    .withColumn('hash_id', F.sha2(F.concat_ws('_', F.col('activity_houe'), F.col('net')), 256).substr(0,15))
)

seismic_activity.display()

In [0]:
table_name = 'lakehouse.`04_gold`.seismic_activity'

In [0]:
if check_delta_table(table_name):
    print(f"{table_name} table exists in the catalog, Updating it")
    delta_merge(seismic_activity, table_name, "hash_id")
else:
    print(f"{table_name} table does not exist in the catalog, Writing it for the first time")
    seismic_activity.write.mode("overwrite").format("delta").option("delta.enableChangeDataFeed", "true").saveAsTable(table_name)

In [0]:
# %sql
# select count(*) from lakehouse.`04_gold`.seismic_activity
# group by hash_id;